<a href="https://colab.research.google.com/github/mmilannaik/BigOCheatSheet/blob/master/FastAPI_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Libraries

In [1]:
!pip install fastapi uvicorn nest-asyncio pyngrok


# Base code

In [2]:
!ngrok config add-authtoken YourToken

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [8]:
from fastapi import FastAPI
import nest_asyncio
from pyngrok import ngrok
import uvicorn

# Allow nested event loops (needed in Colab)
nest_asyncio.apply()

# Create FastAPI app
app = FastAPI()

@app.get("/")
def hello():
    return {'message':'Hello world'}

@app.get('/about')
def about():
    return {'message':'Campusx is an education platform where youcan learn AI'}

# Expose the app to the outside world with ngrok
public_url = ngrok.connect(8000)
print('Public URL:', public_url)

# Run the FastAPI app
uvicorn.run(app, host="0.0.0.0", port=8000)


INFO:     Started server process [509]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


Public URL: NgrokTunnel: "https://195e592fe4ca.ngrok-free.app" -> "http://localhost:8000"
INFO:     167.103.7.98:0 - "GET / HTTP/1.1" 200 OK
INFO:     167.103.7.98:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [509]


# Patient Code

In [3]:
from fastapi import FastAPI,Path,HTTPException,Query
from fastapi.responses import JSONResponse
import nest_asyncio
from pyngrok import ngrok
import uvicorn,json
from pydantic import BaseModel,Field,computed_field
from typing import Annotated,Literal,Optional

# Allow nested event loops (needed in Colab)
nest_asyncio.apply()

# Create FastAPI app
app = FastAPI()

class Patient(BaseModel):
  id: Annotated[str, Field(...,description = 'ID of the patient',examples = ['P001'])]
  name :Annotated[str,Field(...,description = 'Name of the patient')]
  city: Annotated[str, Field(...,description = 'City where the patient living')]
  age: Annotated[int, Field(...,gt=0,lt=120,description ='age of the patient')]
  gender: Annotated[Literal['male','female','others'], Field(...,description = 'Gender of patient')]
  height:Annotated[float, Field(...,gt=0,description = 'Height of patient in mtrs')]
  weight:Annotated[float,Field(...,gt=0,description = 'Weight of patient in kgs')]

  @computed_field
  @property
  def bmi(self)->float:
    bmi = round(self.weight/(self.height**2),2)

  @computed_field
  @property
  def verdict(self)->str:
    if self.bmi <18.5:
      return 'underweight'
    elif self.bmi <25:
      return 'Normal'
    elif self.bmi <30:
      return 'Normal'
    else:
      return 'obese'

class PatientUpdate(BaseModel):
    name: Annotated[Optional[str], Field(default=None)]
    city: Annotated[Optional[str], Field(default=None)]
    age: Annotated[Optional[int], Field(default=None, gt=0)]
    gender: Annotated[Optional[Literal['male', 'female']], Field(default=None)]
    height: Annotated[Optional[float], Field(default=None, gt=0)]
    weight: Annotated[Optional[float], Field(default=None, gt=0)]


def load_data():
  with open('/content/patients.json','r') as f:
    data = json.load(f)
  return data

def save_data(data):
  with open('/content/patients.json','r') as f:
    json.dump(data,f)

@app.get("/")
def hello():
    return {'message':'Patient Managemet System API'}

@app.get('/about')
def about():
    return {'message':'A fully functional API to manage your patient records'}

@app.get('/view')
def view():
  data = load_data()
  return data

@app.get('/patient/{patient_id}')
def view_patient(patient_id :str = Path(..., description='ID of the patient in the DB', example='P001')):
  # load all patients
  data = load_data()
  patient_id = patient_id.upper()
  if patient_id in data:
    return data[patient_id]
  return HTTPException(status_code= 404,detail='patient not found')

@app.get('/sort')
def sort_patients(sort_by: str = Query(...,description = 'Sort on the bais of height, weight or bmi'),order: str = Query('asc',description ='sort in asc or desc order')):
  valid_fields = ['height','weight','bmi']
  if sort_by not in valid_fields:
    raise HTTPException(status_code = 400,detail = f'Invalid field selected from {valid_fields}')
  if order not in ['asc','desc']:
    raise HTTPException(status_code = 400,detail = f'Invalid field selected between asc and desc')
  data = load_data()

  sort_order = True if order =='desc' else False
  sorted_data = sorted(data.values(),key = lambda x:x.get(sort_by,0),reverse=sort_order)

  return sorted_data

@app.post('/create')
def create_patient(patient: Patient):
  # load existing data
  data = load_data()
  # check if the patient already exists
  if patient.id in data:
    raise HTTPException(status_code=400, detail = 'Patient already exists')

  # new patient add to db
  data[patient.id] = patient.model_dump(exclude=['id'])

  # save data into json file
  save_data(data)

  return JSONResponse(status_code=201,content={'message':'patient created sucessfully'})

@app.put('edit/{patien_id}')
def update_patient(patient_id: str, patient_update:PatientUpdate):
  data = load_data()

  if patient_id not in data:
    raise HTTPException(status_code=404,detail ='Patient not found')
  existing_patient_info = data[patient_id]

  updated_patient_info = patient_update.model_dump(exclude_unset=True) # only filled item willbe sent
  for key,value in updated_patient_info.items( )

# Expose the app to the outside world with ngrok
public_url = ngrok.connect(8000)
print('Public URL:', public_url)

# Run the FastAPI app
uvicorn.run(app, host="0.0.0.0", port=8000)



/tmp/ipython-input-3-3878596822.py:64: DeprecationWarning: `example` has been deprecated, please use `examples` instead
  def view_patient(patient_id :str = Path(..., description='ID of the patient in the DB', example='P001')):
INFO:     Started server process [276]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


Public URL: NgrokTunnel: "https://f9a3e8b9da3c.ngrok-free.app" -> "http://localhost:8000"
INFO:     167.103.7.98:0 - "GET / HTTP/1.1" 200 OK
INFO:     167.103.7.98:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     167.103.7.98:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     167.103.7.98:0 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     167.103.7.98:0 - "POST /create HTTP/1.1" 400 Bad Request
INFO:     167.103.7.98:0 - "POST /create HTTP/1.1" 500 Internal Server Error


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/protocols/http/h11_impl.py", line 403, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/middleware/proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/fastapi/applications.py", line 1054, in __call__
    await super().__call__(scope, receive, send)
  File "/usr/local/lib/python3.11/dist-packages/starlette/applications.py", line 112, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.11/dist-packages/starlette/middleware/errors.py", line 187, in __call__
    raise exc
  File "/usr/local/lib/python3.11/dist-packages/starlette/middleware/errors.py",